In [ ]:
import boto3

from urllib.request import urlopen
from PIL import Image 

import pandas as pd
import json

In [ ]:
# df = pd.read_csv('../data/data_including_subscriber_and_thumbnail.csv')
# df = df[['video_id', 'views', 'subscriber_count', 'thumbnail']]
# df

In [ ]:
# df = df[df.views > 1000]
# df

In [ ]:
df = pd.read_csv('../data/face_dectection_data.csv')

In [ ]:
# Initialize

# df['people_num'] = -1
# df['age'] = -1
# df['width'] = -1
# df['height'] = -1
# df['emotion'] = ''
# df['gender'] = ''

In [ ]:
def detect_faces(url):

    client=boto3.client('rekognition')

    response = client.detect_faces(
        Image={'Bytes': urlopen(url).read()},
        Attributes=['ALL']
    )
    
    people_num = len(response['FaceDetails'])
    
    # 감지된 얼굴이 없는 경우
    if people_num == 0:
        return [int(people_num), -1, -1, -1, '', '']
    
    # 가장 큰 얼굴 하나만 데이터로 저장
    max_height = 0
    
    for faceDetail in response['FaceDetails']:
        # size
        height = faceDetail['BoundingBox']['Height']
        
        if height > max_height:
            max_height = height
        else: 
            continue
        
        width = faceDetail['BoundingBox']['Width']
        
        # age는 예상 나이 범위의 (low+high)/2
        age = (faceDetail['AgeRange']['Low'] + faceDetail['AgeRange']['High']) / 2
        
        # size
        width = faceDetail['BoundingBox']['Width']
        height = faceDetail['BoundingBox']['Height']
        
        # emotion
        emotion_confidence = 0
        emotion = ''
        
        for item in faceDetail['Emotions']:
            if item['Confidence'] > emotion_confidence:
                emotion_confidence = item['Confidence']
                emotion = item['Type']
        
        # gender
        gender = faceDetail['Gender']['Value']
    
    
    data = [int(people_num), int(age), width, height, emotion, gender]
    return data

In [ ]:
def get_data(start_index, end_index):
    if end_index > len(df):
        end_index = len(df)
    for index in range(start_index, end_index):
        url = df['thumbnail'][index]
        data = detect_faces(url)
        df.loc[index, ['people_num', 'age', 'width', 'height', 'emotion', 'gender']] = data

In [ ]:
get_data(0,10000)

df

In [ ]:
df.to_csv('../data/face_dectection_data.csv', index=False)